# Setting colab

In [6]:
#drive mount
#구글 드라이브를 colab에서 사용할 수 있도록 마운트
import os, sys 
from google.colab import drive 
drive.mount('/content/mnt') 


Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [7]:
#file이 저장된 위치로 working directory 변경
%cd /content/mnt/MyDrive/Colab Notebooks/AI_SA_hw


/content/mnt/MyDrive/Colab Notebooks/AI_SA_hw


In [8]:
#file list of current working directory
!ls

Corpus	data.zip  main.ipynb  output.txt  Vector_Corpus


In [ ]:
#unzip data file
!unzip data.zip

Archive:  data.zip


KeyboardInterrupt: ignored

#TF_IDF

In [9]:
import os
import math
import pandas as pd

'''
searching .txt files' path
'''
def search(dirname, files):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, files)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.txt':
                    files.append(full_filename)
    except PermissionError:
        pass

'''
mapping features to indices
return : {"갑/NNG":0, "을/NNG":1, ... }
'''
def idcs_of_features(features_file):
    features = {}
    with open(features_file, 'r', encoding='utf=8') as f:
        i = 0
        while True:
            line = f.readline()
            if not line:
                break
            key = line.split()[0]
            features[key] = i
            i += 1
    return features

'''
calculate TF for a document(file)
return : [0, 0, 1, ... ]
'''
def calculate_TF(file, features):
    TF = [0.] * len(features)
    words = []
    with open(file, 'r', encoding='utf-8') as f:
        while True:
            line = f.readline()
            if line is not '\n' and line is not '':
                tokens = list(map(str, line.split('\t')))
                if '+' in tokens[1]:
                    text = tokens[1].split('+')
                    for tx in text:
                        words += [tx.replace('\n', '')]
                else:
                    words += [tokens[1].replace('\n', '')]
            if not line:
                break
    # count the words respectively
    for word in words:
        if word in features.keys():
            TF[features[word]] += 1
    return TF

'''
calculate IDF for all of documents
return : [4, 3, ... ]
'''
def calculate_IDF(documents, features, TF):
    D = len(documents)
    IDF = [0] * len(features)
    # count IDF
    for i in range(D):
        # count the IDF using train data set
        if 'Input_Data' in documents[i]:
            for key in features.keys():
                if TF[i][features[key]] > 0:
                    IDF[features[key]] += 1
    return IDF

'''
calculate normalized TF-IDF for a file
return : [0., 0.0033242.., 0., 0., ... ]
'''
def TF_IDF_Calculation(features, TF, IDF):
    TFIDF = [0] * len(features)
    TFIDF_sum = 0
    for key in features.keys():
        TFIDF_sum += (TF[features[key]]*IDF[features[key]])**2
    for key in features.keys():
        if TFIDF_sum == 0:  # if TF or IDF is zero, then TFIDF_sum is changed to a very small value
            TFIDF_sum = 0.00000000000000000000000001
        TFIDF[features[key]] = (TF[features[key]]*IDF[features[key]]) / math.sqrt(TFIDF_sum)
    return TFIDF


Fill the below cell

In [31]:
from collections import Counter

#count vocab of all documnet and save file in "output.txt"(number of lines in output.txt is 10000)
#output txt is split by \t ex) word \t numOfDocumentIncludeWord
#output.txt:
#./SF	4890
#ㄴ/ETM	3154
#이/VCP	3121
def calculate_DF_top_10000(DF_table_file_path, file_dirs):
    new_list = []
    # Corpus 폴더 내 모든 파일로부터 저장된 string 읽어오기
    # tab 이후의 문장을 '+' 기준으로 split
    # new_list에 모든 word 저장됨.
    for i in range(len(file_dirs)):
        with open(file_dirs[i], encoding='utf8') as  f:
            for line in f:
                line = line.rstrip('\n')
                index = line.find('\t')
                line = line[index+1:]
                my_list = line.split('+')
                new_list.extend(my_list)

    # 각 word별 빈도 수 구하기
    wordCount = {}
    for word in new_list:
        wordCount[word] = wordCount.get(word,0)+1

    # 알파벳 기준으로 먼저 sorting 후, 빈도 수 기준으로 sorting
    wordCount = sorted(sorted(wordCount.items(),key=lambda x:x[0], reverse=False ),key=lambda x:x[1], reverse=True)
    wordCount = dict(wordCount[:10000])
    
    keys = list(wordCount.keys())
    values = list(wordCount.values())
    
    # output.txt에 결과 저장
    with open(DF_table_file_path, 'w') as output:
        for i in range(len(keys)):
            output.write(keys[i] + "\t%d\n" %values[i])
    return None

#calculate of idf list of using "output.txt" and feature set({"갑/NNG":0, "을/NNG":1, ... })
#return idf list of all feature index
#DF_table_file_path: output.txt
#feature_set: index dictionary ({"갑/NNG":0, "을/NNG":1, ... })   len(feature_set) == 10000
#file_dir: all train file paths for calculate of all document number(len(file_dirs))
#return : [0.49191975230946466, 0.9303280378735083, 0.9408427117734446, ... , ]
#len return is 10000
def calculate_IDF(DF_table_file_path, feature_set, file_dirs): 
    docNum = len(file_dirs)
    key_list = []
    value_list = []

    IDF_table = [0] * len(feature_set)
        
    with open(DF_table_file_path, encoding='utf8') as  f:
        for line in f:
            line = line.rstrip('\n')
            split = line.split('\t')
            key_list.append(split[0])
            value_list.append(int(split[1]))
        dic = dict(zip(key_list, value_list))
        print("Dic: ", dic)
    
    import math
    i=0
    for(term, term_frequency) in dic.items():
        term_IDF = math.log(float(docNum)/term_frequency)
        #IDF_table.insert(i,(term_IDF, term) )
        IDF_table[i]= term_IDF 
        i = i+1

    print(len(IDF_table))
    
    return IDF_table

#calculate TF vector of file
#read fild_dir's file and count vocab and return TF vector
#return: [0.0, 0.6448542564478883, 0.6521424730161364, 0.7704833585056773, ... , 0]
#len return is 10000
def calculate_TF_vector(file_dir, feature_set):
    tf = [0] * len(feature_set) 
    new_list = []

    with open(file_dir, encoding='utf8') as  f:
            for line in f:
                line = line.rstrip('\n')
                index = line.find('\t')
                line = line[index+1:]
                my_list = line.split('+')
                new_list.extend(my_list)


    i=0
    for(term, term_frequency) in feature_set.items():
        tf[i] = new_list.count(term)
        i = i+1

    return tf



In [ ]:
import json

DF_file_path = "output.txt"
TF_IDF_Data_path = "Vector_Corpus"

#set data path of train, validation, test
train_data_paths = []
search("./Corpus/Input_Data", train_data_paths)
val_data_paths = []
search("./Corpus/Val_Data", val_data_paths)
test_data_paths = []
search("./Corpus/Test_Data", test_data_paths)

#calculate IDF tabel using train data
#calculate_DF_top_10000(DF_file_path, train_data_paths) #count number of document that include vocab of top 10000
feature_set = idcs_of_features(DF_file_path) #make index dictionary of top 1000 words {"갑/NNG":0, "을/NNG":1, ... }
IDF_table = calculate_IDF(DF_file_path, feature_set, train_data_paths)

print(IDF_table)
print(feature_set)

#calculate TF of each document and save IDF vector in TF_IDF_Data_path
all_paths = train_data_paths + val_data_paths + test_data_paths
for i in all_paths:
  TF = calculate_TF_vector(i, feature_set) #calculate TF vector of each file
  TF_IDF = [a*b for a,b in zip(TF,IDF_table)]# calculate TF_IDF vector(TF*IDF)

  #save TF_IDF vector file in Vector_Corpus folder
  new_path = i.split('/')
  new_path[1] = TF_IDF_Data_path
  new_path = '/'.join(new_path)
  print(new_path)
  with open(new_path,"w",encoding="utf8") as f:
    json.dump(TF_IDF,f)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
./Vector_Corpus/Input_Data/POS/POS_1965.txt
./Vector_Corpus/Input_Data/POS/POS_708.txt
./Vector_Corpus/Input_Data/POS/POS_1573.txt
./Vector_Corpus/Input_Data/POS/POS_1185.txt
./Vector_Corpus/Input_Data/POS/POS_2975.txt
./Vector_Corpus/Input_Data/POS/POS_3480.txt
./Vector_Corpus/Input_Data/POS/POS_2586.txt
./Vector_Corpus/Input_Data/POS/POS_243.txt
./Vector_Corpus/Input_Data/POS/POS_59.txt
./Vector_Corpus/Input_Data/POS/POS_439.txt
./Vector_Corpus/Input_Data/POS/POS_3999.txt
./Vector_Corpus/Input_Data/POS/POS_1732.txt
./Vector_Corpus/Input_Data/POS/POS_162.txt
./Vector_Corpus/Input_Data/POS/POS_1103.txt
./Vector_Corpus/Input_Data/POS/POS_2878.txt
./Vector_Corpus/Input_Data/POS/POS_2997.txt
./Vector_Corpus/Input_Data/POS/POS_1382.txt
./Vector_Corpus/Input_Data/POS/POS_2743.txt
./Vector_Corpus/Input_Data/POS/POS_3089.txt
./Vector_Corpus/Input_Data/POS/POS_3787.txt
./Vector_Corpus/Input_Data/POS/POS_3696.txt
./Vector_Corpus/Input_Data/POS/POS_2950.txt
./

#Sentiment Analysis with Deep learning

dataset code

In [33]:
from torch.utils.data import Dataset
import json

#get data in Vector corpus
class MyDataset(Dataset):
  def __init__(self, data_paths):
    super(MyDataset, self).__init__()
    self.data_paths = data_paths
  
  def __len__(self):
    return len(self.data_paths)
  
  def __getitem__(self, idx):
    with open(self.data_paths[idx],"r",encoding="utf8") as f:
      tf_idf_vector = list(json.load(f))
    tf_idf_vector = torch.Tensor(tf_idf_vector)

    if "NEG" in self.data_paths[idx]:
      label = torch.Tensor([0])
    else:
      label = torch.Tensor([1])
    
    return tf_idf_vector, label

model code

In [34]:
from torch import nn

#sentiment analysis model with 3 layer DNN
class MyModel(nn.Module):
  def __init__(self, input_dim, num_label):
    super(MyModel, self).__init__()

    self.ANN1 = nn.Linear(10000,1000)
    self.gelu = nn.GELU()
    self.ANN2 = nn.Linear(1000,100)
    self.classifier = nn.Linear(100,2)

  def forward(self, input, label):
    output = self.ANN1(input)
    output = self.gelu(output)
    output = self.ANN2(output)
    output = self.gelu(output)
    output = self.classifier(output)

    loss_fct = nn.CrossEntropyLoss()
    loss = loss_fct(output.view(-1, 2), label.view(-1))

    return output, loss

In [35]:
#calculate F1 score with predict and groundthruth
def calculate_F1_score(pred,gold):
  TP, FP, TN, FN = 0, 0, 0, 0
  for p,g in zip(pred,gold):
    if p == 1 and g == 1:
      TP = TP+1
    if p == 0 and g == 1:
      FN = FN+1
    if p == 1 and g == 0:
      FP = FP+1
    if p == 0 and g == 0:
      TN = TN+1

  precision = float(TP)/float(TP+FP)
  recall = float(TP)/float(TP+FN)
  
  f1_score = 2*precision*recall/(precision+recall)
  return precision, recall, f1_score


main function of Sentiment analysis

In [36]:
'''
searching .txt files' path
'''
def search(dirname, files):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, files)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.txt':
                    files.append(full_filename)
    except PermissionError:
        pass

import torch
from torch.utils.data import DataLoader, RandomSampler
import torch.optim as optim
from fastprogress.fastprogress import master_bar, progress_bar
import copy

#set data path of train, validation, test
train_data_paths = []
search("./Vector_Corpus/Input_Data", train_data_paths)
val_data_paths = []
search("./Vector_Corpus/Val_Data", val_data_paths)
test_data_paths = []
search("./Vector_Corpus/Test_Data", test_data_paths)

#make dataset
train_dataset = MyDataset(train_data_paths)
val_dataset = MyDataset(val_data_paths)
test_dataset = MyDataset(test_data_paths)


#porting dataset in dataloader
train_sampler = RandomSampler(train_dataset)
train_dataLoader = DataLoader(train_dataset, sampler=train_sampler, batch_size=128)
val_sampler = RandomSampler(val_dataset)
val_dataLoader = DataLoader(val_dataset, sampler=val_sampler, batch_size=128)
test_sampler = RandomSampler(test_dataset)
test_dataLoader = DataLoader(test_dataset, sampler=test_sampler, batch_size=len(test_dataset))

model = MyModel(10000,2).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0
bset_epoch = 0

EPOCH = 10
for epoch in range(EPOCH):
  print("------------------EPOCH:{}------------------".format(epoch+1))
  print("train prograss")
  epoch_iterator = progress_bar(train_dataLoader)

  #train
  model.train()
  train_predict = torch.tensor([]).cuda()
  for step, (data, label) in enumerate(epoch_iterator):
    optimizer.zero_grad()
    data = data.cuda()
    label = label.long().cuda()

    output, loss = model(data, label)

    indices = torch.argmax(output,dim=1)
    train_predict = torch.cat([train_predict,torch.eq(indices, label.view(-1))])
    loss.backward()
    optimizer.step()
  acc = torch.sum(train_predict).item()/train_predict.nelement()
  print('train accuracy=', acc)

  #validation
  print("\n\nvalidation prograss")
  epoch_iterator = progress_bar(val_dataLoader)

  model.eval()
  val_predict = torch.tensor([]).cuda()
  for step, (data, label) in enumerate(epoch_iterator):
    data = data.cuda()
    label = label.long().cuda()

    output, loss = model(data, label)

    indices = torch.argmax(output,dim=1)
    val_predict = torch.cat([val_predict,torch.eq(indices, label.view(-1))])
  val_acc = torch.sum(val_predict).item()/val_predict.nelement()
  print('validation accuracy=', val_acc)
  
  #save best model
  if val_acc > best_val_acc:
    best_param = copy.deepcopy(model.state_dict())
    best_val_acc = val_acc
    best_epoch = epoch

  print("-------------------------------------------\n\n".format(epoch))

print("------------------result--------------------")
print("best epoch is ",best_epoch+1)
print("load best epoch's parameter in model")

#load best model
model.load_state_dict(best_param)
model.eval()

#test
epoch_iterator = progress_bar(test_dataLoader)
test_predict = torch.tensor([]).cuda()
for step, (data, label) in enumerate(epoch_iterator):
  data = data.cuda()
  label = label.long().cuda()

  output, loss = model(data, label)

  predict = torch.argmax(output,dim=1)
  test_predict = torch.cat([test_predict,torch.eq(predict, label.view(-1))])
test_acc = torch.sum(test_predict).item()/test_predict.nelement()
print('test accuracy=', test_acc)
label_list = label.view(-1).tolist()
predict_list = predict.view(-1).tolist()
precision, recall, f1_score = calculate_F1_score(predict_list, label_list)
print('test precision=', precision)
print('test recall=', recall)
print('test f1_score=', f1_score)



------------------EPOCH:1------------------
train prograss


train accuracy= 0.7274659332416552


validation prograss


validation accuracy= 0.7617617617617618
-------------------------------------------


------------------EPOCH:2------------------
train prograss


train accuracy= 0.9132391548943618


validation prograss


validation accuracy= 0.7647647647647647
-------------------------------------------


------------------EPOCH:3------------------
train prograss


...

train accuracy= 0.9969996249531191


validation prograss


validation accuracy= 0.7627627627627628
-------------------------------------------


------------------EPOCH:10------------------
train prograss


train accuracy= 0.9974996874609326


validation prograss


validation accuracy= 0.7577577577577578
-------------------------------------------


------------------result--------------------
best epoch is  3
load best epoch's parameter in model


test accuracy= 0.789
test precision= 0.8016359918200409
test recall= 0.7747035573122529
test f1_score= 0.7879396984924623
